# **Procesamiento del habla**

**VERSION 1.0**

## Funcionalidades versionadas:

VER 1.0
- Uso de dataset filtrado por frutas y verduras
- Entrada de consulta para un producto por vez
- Recomendación para el cliente:
    - Algoritmo de ordenamiento basado en preferencias del usuario (dentro de la estructura gramatical):
        - Características particulares del producto.
            - Basado en el precio.
            - Generales

*VER 1.3:*
- Entrada de consulta para más de un producto.
- Recomendaciones basadas en análisis de sentimiento.


## Estructura de sub-etapas
--- 
```
├── Reconocimiento del lenguaje
│ ├── Captura de texto
│ └── Captura de audio
├── Procesamiento del lenguaje
│ ├── Preprocesamiento de texto
│ └── Análisis de texto
├── Gestión de respuesta
│ ├── Gestión del estado
│ ├── Motor de respuesta
│ └── Personalización
├── Texto a voz
│ └── Conversión de texto a voz

```

En esta etapa, se utilizaran varias librerías clave para diferentes etapas del proceso de recomendación basado en la entrada del usuario. Las principales librerías empleadas serán: SpeechRecognition para la transcripción de voz a texto, spaCy para el procesamiento y análisis del texto, y Google Text-to-Speech (gTTS) para convertir texto a voz.

El proceso comenzará con la captura de la entrada del usuario a través de texto directo o por voz mediante SpeechRecognition, que transformará la voz en texto. Este texto será procesado con spaCy para extraer y lematizar tokens, filtrando específicamente adjetivos, sustantivos y proposiciones. Simultáneamente, se analizarán los productos en nuestro dataframe mediante spaCy, lematizando y filtrando tokens relevantes de la misma manera.

Luego, compararemos los tokens procesados de la entrada del usuario con los tokens procesados de los productos, para evaluar la similitud. Durante esta comparación, también verificaremos si algún token es sinónimo de adjetivos relacionados con el precio del producto. Con esta información, generaremos dos listas de productos: una ordenada por la coincidencia general de los tokens y otra por el precio, en orden ascendente.

Finalmente, ambas listas se integrarán en una recomendación que se almacenará en un diccionario, vinculada al índice del producto en la tabla original. Esta recomendación se convertirá a voz mediante Google Text-to-Speech y se entregará al usuario dentro de un diccionario, proporcionando una experiencia interactiva y personalizada.

### Librerias y paquetes

#### *Entorno virtuales*

In [1]:
# Create and activate virtual env for speech processing stage
!python3 -m venv proc_habla
!source proc_habla/bin/activate

#### *Instalacion*

In [2]:
# Librerías de preprocesamiento
!pip3 install scipy

# Librerías de reconocimiento de voz
!pip3 install SpeechRecognition

# Librerías de generación de voz
!pip3 install gtts

# Librerías de análisis lingüístico
!pip3 install spacy

In [3]:
# Descargo el modelo de la libreria spacy pre-entrenado con corpus en Español
!python3 -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 292.0 kB/s eta 0:00:0000:0100:02
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


#### *Importacion*

In [4]:
# Librerías para reconocimiento de voz y procesamiento de audio
import speech_recognition as sr

# Librerías para manipulación de archivos y compresión
import os
import io

# Librerías para procesamiento de datos y análisis
import numpy as np
import pandas as pd
import spacy
from spacy import displacy
import re

# Librería para generación de voz
from gtts import gTTS

# Importación de funciones comunes a otros cuadernos
from funciones_comunes import common_functions

### Funciones

#### Captura de texto de entrada

In [5]:
def entrada_texto():
    texto = input('Ingresa brevemente el producto que deseas comprar y sus características: ')
    return texto

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [6]:
def entrada_voz():
    recognizer = sr.Recognizer()  # Se inicializa el objeto de la clase speech recognition con el metodo recognizer y almacena en la variable 'recognizer'
    mic = sr.Microphone()  # Configuración del micrófono como fuente de audio

    with mic as source:
        recognizer.adjust_for_ambient_noise(source, duration=1)
        # Ajuste de sensibilidad y tiempo de pausa
        recognizer.energy_threshold = 300  # Ajustar según el entorno
        recognizer.pause_threshold = 1.0  # Ajustar según necesidad
        print("Por favor, habla ahora.")  # Indicación para el usuario
        audio = recognizer.listen(source)  # Escucha y captura del audio mientras escuche entrada de voz continua

    try:
        texto = recognizer.recognize_google(audio, language='es-ES')  # Reconocimiento de voz utilizando Google
    except sr.UnknownValueError:
        print("No se pudo entender el audio")  # Manejo de error en caso de audio no reconocido
    except sr.RequestError as e:
        print("Error al conectarse con el servicio de Google: {0}".format(e))  # Manejo de error en la conexión con Google

    return texto  # retorna el texto reconocido por audio

#### Comprovacion de sinonimos

In [7]:
def verifica_sinonimo_precios(tokens, sinonimos):
    '''
    Esta función verifica si dentro de la lista de tokens proporcionada como primer parámetro
    se encuentra un sinónimo o derivación morfológica de alguna de las palabras dentro del diccionario de sinónimos.

    Args:
    'tokens': Recibe los tokens lematizados de la entrada del usuario.
    'sinonimos': Diccionario de sinónimos para las referencias.

    Retorno:
    Esta función retorna como resultado 'True' o 'False', según corresponda.
    '''
    # Convertir la lista de tokens a un conjunto para mejorar la eficiencia de búsqueda
    tokens_set = set(tokens)
    print(tokens_set)
    # Recorrer todos los sinónimos en el diccionario
    for sinonimos_list in sinonimos.values():
        for sinonimo in sinonimos_list:
            if sinonimo in tokens_set:
                return True
                
    return False

### Lectura de datos

#### *Datos fuentes*

In [8]:
# Defino el diccionario donde se almacenara la informacion de entrada y salida de la consulta
consulta_dict = {}

In [9]:
# Lectura y almacenado del dataframe
path_csv_rutas_verduras = '../../datos/procesaodos/VerdurasporSupermercado.csv'
df_frutas_verduras = pd.read_csv(path_csv_rutas_verduras)

## 1. Reconocimiento del Habla

___

### a. Captura de entradas de texto

In [10]:
# Conexión con la API de FastAPI
# Aquí iría la conexión con un script encargado de conectar con la API de FastAPI

In [11]:
# Función tentativa para recibir la entrada de texto desde FastAPI
# funcion definida al comienzo del notebook

### b. Captura de Audio

#### Reconocimiento del Habla (ASR - Automatic Speech Recognition):

In [12]:
# funcion definida al comienzo del notebook

### c. Identificar tipo de entrada

In [13]:
# Tipo de entrada específica pasada por fast-api
entrada_tipo = ''

# ejecucion de funciones de entrada segun corresponda la modalidad / tipo de la misma
if entrada_tipo == 'audio':
    texto = entrada_voz()  # Si la entrada es de tipo 'audio', se utiliza la función para recibir entrada de voz
elif entrada_tipo == 'texto':
    texto = entrada_texto()  # Si la entrada es de tipo 'texto', se utiliza la función para recibir entrada de texto
else:
    # Se define texto de prueba en caso de que no se espesifique el tipo de entrada
    texto = 'Quiero tomates rojos, grandes y economicos'
    pass

## 2. Procesamiento del Lenguaje Natural

___

In [14]:
# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

### Preprocesamiento de texto


In [15]:
# Lista de abreviaturas y formas no abreviadas de unidades de medida de peso
'''

# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',
                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',
                        #'toneladas', 'libras', 'onzas', 'quintales']

'''

"\n\n# unidades_medida_peso = ['g', 'gr', 'kg', 'mg', 'µg', 't', 'lb', 'oz', 'cwt',\n                        #'gramos', 'kilogramos', 'miligramos', 'microgramos',\n                        #'toneladas', 'libras', 'onzas', 'quintales']\n\n"

#### *Normalización del texto*

In [16]:
# texto procesado por la funcion 'common_functions.procesar_texto'
texto_analizado = common_functions.procesar_texto(texto, graph=True, nlp = nlp)

#### *Tokenización*

In [17]:
# tokens del texto procesado
tokens = texto_analizado['tokens']

#### *Eliminación de stop words*


In [18]:
# Tokens filtrados
tokens_filtrados = texto_analizado['filtered_tokens']

#### *Lematización*

In [19]:
# lemas de los tokens filtrados
lemmas_filtrados = texto_analizado['lemmas']

#### *Etiquetado de estructuras gramaticales*

In [20]:
# etiquetas gramaticales de los tokens filtrados
pos_tags_filtrados = texto_analizado['pos_tags']

### Análisis de texto


Deteccion de sinonimos de preferencias basadas en el precio

In [21]:
# Lista de sinónimos para las referencias
sinonimos = {
    "económico": ["barato", "asequible", "económico"],
    "barato": ["económico", "asequible", "barato"],
    "asequible": ["económico", "barato", "asequible"],
    "precio": ["coste", "precio"],
    "coste": ["precio", "coste"]
}

# Verificar si hay algún sinónimo o derivación en los tokens lematizados
consulta_precio = verifica_sinonimo_precios(lemmas_filtrados, sinonimos=sinonimos)

{'economico', 'rojo', 'tomate'}


Detectar presencia de palabras de productos lematizadas dentro de dataframe

In [22]:
# carga del conjunto de datos en dataframe
df_frutas_verduras['producto_tokens_lemmas'] = df_frutas_verduras['Producto'].apply(lambda x: common_functions.procesar_texto(x, nlp = nlp)['lemmas'])

In [23]:
# Diccionario de coincidencias entre tokens lematizados de la columna y la entrada del usuario
dict_count_coincidences = {}

Algoritmos predefinidos de clasificacion según las intenciones y preferencias detectadas.


In [24]:
# Algoritmo para contar coincidencias entre tokens lematizados de la entrada del usuario y la columna/fila del DataFrame
for index, row in df_frutas_verduras.iterrows():  # Iterar sobre las filas del DataFrame
    for token_producto in row['producto_tokens_lemmas']:  # Iterar sobre los tokens lematizados de la columna 'producto_tokens_lemmas'
        if token_producto in lemmas_filtrados:  # Verificar si el token lematizado está en la lista de lemas filtrados de la entrada del usuario
            if token_producto not in dict_count_coincidences:  # Si el token no está en el diccionario de coincidencias
                dict_count_coincidences[index] = 1  # Agregar el token al diccionario con el valor 1
            else:  # Si el token ya está en el diccionario de coincidencias
                dict_count_coincidences[index] += 1  # Incrementar el valor del token en el diccionario

In [25]:
# Se transforma el diccionario a una seria de python para luego poder ordenarla por valor
series_ordered_count_coincidences = pd.Series(dict_count_coincidences)

# Asegurar que series_ordered_count_coincidences es una Serie ordenada
series_ordered_count_coincidences = series_ordered_count_coincidences.sort_values(ascending=False)

Análisis de Sentimiento: Determinar la emoción o tono del texto.


In [26]:
# se desarrollara en futuras versiones

Detección de Intenciones (Intent Detection): Identificar la intención del usuario utilizando modelos como BERT, GPT, RASA, etc.


In [27]:
# se desarrollara en futuras versiones

## 3. Gestión del Diálogo

___

### Módulo de Gestión de Estado:

In [28]:
# Asignación de valores al diccionario de consulta
consulta_dict['id_consulta'] = None  # ID de la consulta (aún no asignado)
consulta_dict['id_cliente'] = None  # ID del cliente (aún no asignado)
consulta_dict['formato_consulta'] = entrada_tipo  # Formato de la consulta (aún no especificado)
consulta_dict['entrada_usuario'] = texto  # Transcripción del audio (aún no disponible)
consulta_dict['entrada_lemas_filtrados'] = lemmas_filtrados  # Lemas filtrados de la entrada del usuario
consulta_dict['dict_producto_indice_considencias'] = dict_count_coincidences  # Diccionario de productos y sus coincidencias
consulta_dict['card_recomendacion'] = None  # Tarjeta de recomendación (aún no generada)
audio_recomendacion = None  # Audio de la recomendación (aún no generado)

### Motor de Respuesta:


Generación de Respuestas: Uso de los algoritmos predefinidos de clasificacion según las intenciones y preferencias detectadas.


In [29]:
# Limpiando la columna 'Precio': eliminando el signo de dólar, eliminando los puntos del millar y reemplazando la coma decimal por un punto
df_frutas_verduras['Precio'] = df_frutas_verduras['Precio'].apply(common_functions.limpiar_signo_peso)

In [31]:
# DataFrame filtrado por columnas de interes
df_productos_filtrados = df_frutas_verduras.loc[series_ordered_count_coincidences.index, ['Producto', 'Supermercado', 'Precio']]

# Se comprueba si hay algun token con relacion los sinonimos de referencia
# DataFrame filtrado, acortado y ordenado por precio
if consulta_precio:
    df_productos_filtrados = df_productos_filtrados.sort_values(by='Precio')
    df_productos_filtrados = df_productos_filtrados[:5]
else:
    # DataFrame filtrado, acortado a 5 resultados
    df_productos_filtrados = df_productos_filtrados[:5]

In [32]:
# Diccionario para almacenar detalles de cada uno de los productos
diccionario_productos_precio = {}

In [30]:
# Inicialización de la recomendación
recomendacion = "Los productos recomendados basados en su consulta y características mencionadas son: \n"

In [33]:
# Iterar sobre los productos filtrados y ordenados
for count, (x, producto_filtrado_info) in enumerate(df_productos_filtrados.iterrows(), start=1):
    # Añadir detalles del producto al diccionario
    diccionario_productos_precio[x] = {
        'Posicion': count,
        'Producto': producto_filtrado_info['Producto'],
        'Supermercado': producto_filtrado_info['Supermercado'],
        'Precio': producto_filtrado_info['Precio']
    }
    # Añadir recomendación a la lista
    recomendacion += f'\n En la posición {count}: {producto_filtrado_info["Producto"]} en {producto_filtrado_info["Supermercado"]} a {producto_filtrado_info["Precio"]} pesos'


In [34]:
# Redondear números en la recomendación (si es necesario)
recomendacion = common_functions.redondear_numeros(recomendacion)

In [35]:
# Asignar la recomendación al diccionario de consulta
consulta_dict['card_recomendacion'] = recomendacion

In [36]:
consulta_dict['card_recomendacion']

'Los productos recomendados basados en su consulta y características mencionadas son: \n\n En la posición 1: Snack manzana roja deshidratada Frutty 18 g. en Carrefour a 535 pesos\n En la posición 2: Pera roja x kg. en Carrefour a 2290 pesos\n En la posición 3: Tomates secos 70 g. en Carrefour a 2229 pesos\n En la posición 4: Snack manzana roja deshidratada Frutty 18 g. en Carrefour a 535 pesos\n En la posición 5: Pera roja x kg. en Carrefour a 2290 pesos'

### Personalización y Contexto: Adaptar las respuestas en función del historial del usuario


In [38]:
# se desarrollara en futuras versiones

## 4. texto a voz

___

### Conversión de texto a Voz (TTS)


In [39]:
# google text to speech tomará la variable recomendacion como el texto a convertir en voz y establecerá el idioma como español ('es')
tts = gTTS(text=recomendacion, lang='es')

Compresion en formato BytesIO

In [40]:
# Guardar el audio en un objeto BytesIO
audio_buffer = io.BytesIO()
tts.write_to_fp(audio_buffer)

In [41]:
# Asegurarse de que el puntero esté al principio del buffer
audio_buffer.seek(0)

0

In [42]:
# Se guarda el audio de respuesta en el diccionario de la consulta
consulta_dict['recomendacion_audio'] = audio_buffer